In [33]:
import pandas as pd
from make_data import *

# Get dict to classify themes

In [2]:
themes = pd.read_csv('data/themes.csv')
themes = themes.drop_duplicates(subset='QUESTION')
themes_dict = get_theme(themes)

In [18]:
themes_dict.items()

dict_items([('Le coeur a ses raisons que la raison ignore', 'Love'), ('Quand-on aime sait-on pourquoi', 'Love'), ("Qu'aime-t-on dans l'amour", 'Love'), ("Est-il raisonnable d'aimer", 'Love'), ('Peut-on aimer son prochain comme soi-même', 'Love'), ('Est-ce réaliste de prétendre pouvoir aimer tous les hommes', 'Love'), ("Dans tout amour n'aime t-on que soi-même", 'Love'), ('Pour aimer autrui faut-il le connaître', 'Love'), ("L'amitié est-elle une forme privilégiée de la connaissance d'autrui", 'Love'), ('Comment différencier amour et amitié', 'Love'), ("Peut-on aimer une oeuvre d'art sans la comprendre", 'Art'), ("Peut-on comprendre une oeuvre d'art sans l'aimer", 'Art'), ("L'art peut-il manifester la vérité", 'Art'), ('Les oeuvres d´art sont-elles des réalités comme les autres', 'Art'), ('L’art transforme-t-il notre conscience du réel', 'Art'), ("La sensibilité aux œuvres d'art demande-t-elle à être éduquée", 'Art'), ('Faut-il des connaissances pour apprécier une œuvre d’art', 'Art'), (

# Create database

## All docs

In [31]:
folder_path = "data/Extraits"
threshold = 85

# This merges chunking results from ALL .docx files in folder_path
merged_dict = chunk_all_docx_in_folder(folder_path, themes_dict, threshold=threshold)

In [37]:
save_path = "data/M0A_train_data.json"

save_dict_to_json(merged_dict, save_path)